In [1]:
pip install opencv-python

In [3]:
cap = cv2.VideoCapture(0)

while(True):
    ret,frame = cap.read()
    cv2.imshow('Capturing Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('y'):
        cap.release()
        cv2.destroyAllWindows()
        break
       


In [15]:
cap.release()

In [1]:
import cv2
import numpy as np

In [28]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

In [29]:
POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [30]:
image_width=600
image_height=600

In [31]:
net = cv2.dnn.readNetFromTensorflow("graph_opt.pb")
threshold=0.2

In [32]:
img = cv2.imread('./img1.png',cv2.IMREAD_UNCHANGED)

In [33]:
photo_height=img.shape[0]
photo_width=img.shape[1]
net.setInput(cv2.dnn.blobFromImage(img, 1.0, (image_width, image_height), (127.5, 127.5, 127.5), swapRB=True, crop=False))

In [34]:
out = net.forward()
out = out[:, :19, :, :] 

In [35]:
assert(len(BODY_PARTS) == out.shape[1])

In [36]:
points = []
for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
    heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
    _, conf, _, point = cv2.minMaxLoc(heatMap)
    x = (photo_width * point[0]) / out.shape[3]
    y = (photo_height * point[1]) / out.shape[2]
    # Add a point if it's confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > threshold else None)

In [37]:
cv2.imshow("hmmm",img)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows()

In [38]:
for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)

    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]

    if points[idFrom] and points[idTo]:
        cv2.line(img, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv2.ellipse(img, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
        cv2.ellipse(img, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

In [39]:
t, _ = net.getPerfProfile()

In [40]:
cv2.imshow("hmmm",img)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows()

In [2]:
def pose_estimation(frame):
    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }
    
    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]
    
    image_width=600
    image_height=600
    
    net = cv2.dnn.readNetFromTensorflow("graph_opt.pb")
    threshold=0.2
    
#     img = cv2.imread('./img1.png',cv2.IMREAD_UNCHANGED)
    img = frame
    
    photo_height=img.shape[0]
    photo_width=img.shape[1]
    net.setInput(cv2.dnn.blobFromImage(img, 1.0, (image_width, image_height), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    
    out = net.forward()
    out = out[:, :19, :, :] 
    
    assert(len(BODY_PARTS) == out.shape[1])
    
    points = []
    for i in range(len(BODY_PARTS)):
            # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

            # Originally, we try to find all the local maximums. To simplify a sample
            # we just find a global one. However only a single pose at the same time
            # could be detected this way.
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (photo_width * point[0]) / out.shape[3]
        y = (photo_height * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > threshold else None)
        
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv2.line(img, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv2.ellipse(img, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(img, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            
    t, _ = net.getPerfProfile()
    
    return img

In [3]:
cap = cv2.VideoCapture(0)

while(True):
    ret,frame = cap.read()
#     cv2.imwrite('./img1.png', frame)
    frame = pose_estimation(frame)
    cv2.imshow('Capturing Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('y'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [ ]:
cap.release()

In [5]:
pose_estimation()